# 💊 Smart Medicine Reminder Agent
## AI-Powered Medicine Reminder System for Indian Families

---

### 🎯 Problem Statement

In India, many elderly people and busy families struggle to remember regular medicine timings. Missed doses can severely affect health, especially for chronic conditions like:
- Blood Pressure (BP)
- Diabetes
- Heart conditions
- Thyroid disorders

Family members often lack time to remind elderly relatives consistently, leading to missed medications and health complications.

---

### 💡 Solution

A **Python-based medicine reminder agent** that:
- Allows users to input medicine schedules with custom messages in **any language** (Hindi, English, or mixed)
- Delivers **spoken reminders** using Text-to-Speech (TTS) at scheduled times
- Provides a **simple, user-friendly interface** suitable for elderly users
- Supports **personalized, family-style messages** for better compliance

---

### 🔑 Key Features

1. **Multilingual Support**: Hindi, English, or any mix
2. **Custom Messages**: Users write their own reminder text
3. **Voice Reminders**: TTS reads the message at scheduled time
4. **Easy Management**: Add, view, edit, and delete reminders
5. **Family-Friendly**: Natural, relatable language (e.g., "Are uncle, aapki BP ki dawai ka time ho gaya hai!")

---

### 📋 This Notebook Demonstrates

- Core agent logic and scheduling mechanism
- User input handling and data storage
- Reminder generation and TTS simulation
- Future mobile app implementation roadmap

---

## 📦 Step 1: Import Required Libraries

In [ ]:
# Core libraries
import datetime
import time
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Dict, Optional
import json

# For Text-to-Speech (TTS)
try:
    from gtts import gTTS
    import os
    TTS_AVAILABLE = True
except ImportError:
    print("⚠️ gTTS not installed. Install with: pip install gtts")
    TTS_AVAILABLE = False

# For playing audio (optional)
try:
    from IPython.display import Audio, display
    AUDIO_AVAILABLE = True
except ImportError:
    AUDIO_AVAILABLE = False

print("✅ Libraries imported successfully!")
print(f"📢 TTS Available: {TTS_AVAILABLE}")
print(f"🔊 Audio Playback Available: {AUDIO_AVAILABLE}")

---

## 🏗️ Step 2: Define the Medicine Reminder Agent Class

In [ ]:
class MedicineReminderAgent:
    """
    Smart Medicine Reminder Agent for Indian Families
    
    Features:
    - Multilingual reminder support (Hindi, English, mixed)
    - Custom personalized messages
    - Voice-based reminders using TTS
    - Easy schedule management
    """
    
    def __init__(self):
        self.reminders: List[Dict] = []
        self.reminder_id_counter = 1
        
    def add_reminder(self, 
                    medicine_name: str, 
                    reminder_time: str, 
                    custom_message: str,
                    frequency: str = "daily") -> Dict:
        """
        Add a new medicine reminder
        
        Args:
            medicine_name: Name of the medicine
            reminder_time: Time in HH:MM format (24-hour)
            custom_message: Personalized reminder message in any language
            frequency: How often (daily, weekly, etc.)
            
        Returns:
            Dictionary containing reminder details
        """
        reminder = {
            'id': self.reminder_id_counter,
            'medicine_name': medicine_name,
            'time': reminder_time,
            'message': custom_message,
            'frequency': frequency,
            'active': True,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        self.reminders.append(reminder)
        self.reminder_id_counter += 1
        
        print(f"✅ Reminder added successfully! (ID: {reminder['id']})")
        return reminder
    
    def view_reminders(self) -> pd.DataFrame:
        """
        View all active reminders as a formatted table
        
        Returns:
            Pandas DataFrame with reminder details
        """
        if not self.reminders:
            print("📭 No reminders scheduled yet.")
            return pd.DataFrame()
        
        active_reminders = [r for r in self.reminders if r['active']]
        
        if not active_reminders:
            print("📭 No active reminders.")
            return pd.DataFrame()
        
        df = pd.DataFrame(active_reminders)
        return df[['id', 'medicine_name', 'time', 'message', 'frequency']]
    
    def delete_reminder(self, reminder_id: int) -> bool:
        """
        Delete a reminder by ID
        
        Args:
            reminder_id: ID of the reminder to delete
            
        Returns:
            True if deleted, False if not found
        """
        for reminder in self.reminders:
            if reminder['id'] == reminder_id and reminder['active']:
                reminder['active'] = False
                print(f"🗑️ Reminder {reminder_id} deleted successfully.")
                return True
        
        print(f"❌ Reminder {reminder_id} not found.")
        return False
    
    def edit_reminder(self, 
                     reminder_id: int, 
                     medicine_name: Optional[str] = None,
                     reminder_time: Optional[str] = None,
                     custom_message: Optional[str] = None) -> bool:
        """
        Edit an existing reminder
        
        Args:
            reminder_id: ID of reminder to edit
            medicine_name: New medicine name (optional)
            reminder_time: New time (optional)
            custom_message: New message (optional)
            
        Returns:
            True if edited, False if not found
        """
        for reminder in self.reminders:
            if reminder['id'] == reminder_id and reminder['active']:
                if medicine_name:
                    reminder['medicine_name'] = medicine_name
                if reminder_time:
                    reminder['time'] = reminder_time
                if custom_message:
                    reminder['message'] = custom_message
                    
                print(f"✏️ Reminder {reminder_id} updated successfully.")
                return True
        
        print(f"❌ Reminder {reminder_id} not found.")
        return False
    
    def generate_tts(self, message: str, filename: str = "reminder.mp3") -> Optional[str]:
        """
        Generate TTS audio from message
        
        Args:
            message: Text to convert to speech
            filename: Output audio filename
            
        Returns:
            Filename if successful, None otherwise
        """
        if not TTS_AVAILABLE:
            print("⚠️ TTS not available. Message would be: " + message)
            return None
        
        try:
            # Auto-detect language (supports Hindi, English, and mixed)
            tts = gTTS(text=message, lang='hi', slow=False)
            tts.save(filename)
            print(f"🔊 Audio generated: {filename}")
            return filename
        except Exception as e:
            print(f"⚠️ TTS Error: {e}")
            return None
    
    def check_and_trigger_reminders(self, current_time: Optional[str] = None) -> List[Dict]:
        """
        Check if any reminders need to be triggered
        
        Args:
            current_time: Time to check (HH:MM format). If None, uses current time.
            
        Returns:
            List of triggered reminders
        """
        if current_time is None:
            current_time = datetime.now().strftime("%H:%M")
        
        triggered = []
        
        for reminder in self.reminders:
            if reminder['active'] and reminder['time'] == current_time:
                triggered.append(reminder)
                print(f"\n⏰ REMINDER TRIGGERED at {current_time}")
                print(f"💊 Medicine: {reminder['medicine_name']}")
                print(f"📢 Message: {reminder['message']}")
                print("─" * 50)
                
                # Generate TTS
                audio_file = self.generate_tts(reminder['message'], 
                                               f"reminder_{reminder['id']}.mp3")
                
                if audio_file and AUDIO_AVAILABLE:
                    display(Audio(audio_file, autoplay=True))
        
        return triggered
    
    def simulate_day(self, times_to_check: List[str]):
        """
        Simulate checking reminders at multiple times throughout a day
        
        Args:
            times_to_check: List of times in HH:MM format
        """
        print("\n🌅 Starting Day Simulation...\n")
        
        for check_time in sorted(times_to_check):
            print(f"\n🕐 Current Time: {check_time}")
            triggered = self.check_and_trigger_reminders(check_time)
            
            if not triggered:
                print("   No reminders at this time.")
        
        print("\n🌙 Day Simulation Complete!\n")
    
    def export_schedule(self, filename: str = "medicine_schedule.json"):
        """
        Export reminder schedule to JSON file
        
        Args:
            filename: Output JSON filename
        """
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.reminders, f, ensure_ascii=False, indent=2)
        print(f"💾 Schedule exported to {filename}")
    
    def import_schedule(self, filename: str = "medicine_schedule.json"):
        """
        Import reminder schedule from JSON file
        
        Args:
            filename: Input JSON filename
        """
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.reminders = json.load(f)
            print(f"📥 Schedule imported from {filename}")
        except FileNotFoundError:
            print(f"❌ File {filename} not found.")

print("✅ MedicineReminderAgent class defined successfully!")

---

## 🚀 Step 3: Initialize the Agent

In [ ]:
# Create an instance of the Medicine Reminder Agent
agent = MedicineReminderAgent()

print("🤖 Medicine Reminder Agent initialized!")
print("📝 Ready to schedule reminders...")

---

## 📝 Step 4: Add Sample Reminders (Multilingual Examples)

Let's add some realistic reminders in Hindi, English, and mixed languages:

In [ ]:
# Example 1: BP Medicine (Hindi)
agent.add_reminder(
    medicine_name="Amlodipine (BP)",
    reminder_time="08:00",
    custom_message="Are uncle, aapki BP ki dawai ka time ho gaya hai! Subah ki dawai le lijiye.",
    frequency="daily"
)

# Example 2: Diabetes Medicine (English)
agent.add_reminder(
    medicine_name="Metformin (Diabetes)",
    reminder_time="08:30",
    custom_message="Good morning! It's time for your diabetes medicine. Please take it with breakfast.",
    frequency="daily"
)

# Example 3: Heart Medicine (Mixed Hindi-English)
agent.add_reminder(
    medicine_name="Atorvastatin (Heart)",
    reminder_time="21:00",
    custom_message="Namaste ji! Raat ki heart medicine lena mat bhooliye. Dinner ke baad le lijiye.",
    frequency="daily"
)

# Example 4: Thyroid Medicine (Hindi)
agent.add_reminder(
    medicine_name="Thyroxine (Thyroid)",
    reminder_time="06:00",
    custom_message="Suprabhat! Aapki thyroid ki dawai ka samay hai. Khali pet lena hai.",
    frequency="daily"
)

# Example 5: Vitamin Supplement (English)
agent.add_reminder(
    medicine_name="Calcium + Vitamin D3",
    reminder_time="20:00",
    custom_message="Evening reminder! Time for your calcium and vitamin D supplement.",
    frequency="daily"
)

print("\n✅ All sample reminders added successfully!")

---

## 📋 Step 5: View All Scheduled Reminders

In [ ]:
print("📅 ACTIVE MEDICINE REMINDER SCHEDULE\n")
print("=" * 80)

schedule_df = agent.view_reminders()
display(schedule_df)

print("\n" + "=" * 80)
print(f"Total Active Reminders: {len(schedule_df)}")

---

## ⏰ Step 6: Simulate Reminder Triggers

Let's simulate a day and see when reminders trigger:

In [ ]:
# Define times to check throughout the day
times_to_simulate = [
    "06:00",  # Early morning - Thyroid medicine
    "07:00",  # No reminder
    "08:00",  # Morning - BP medicine
    "08:30",  # Morning - Diabetes medicine
    "12:00",  # Noon - No reminder
    "14:00",  # Afternoon - No reminder
    "18:00",  # Evening - No reminder
    "20:00",  # Evening - Vitamin supplement
    "21:00",  # Night - Heart medicine
]

# Run the simulation
agent.simulate_day(times_to_simulate)

---

## ✏️ Step 7: Demonstrate Editing a Reminder

In [ ]:
print("📝 EDITING REMINDER DEMONSTRATION\n")

# Edit reminder ID 1 (BP medicine) - change time and message
agent.edit_reminder(
    reminder_id=1,
    reminder_time="08:15",
    custom_message="Are uncle, BP ki dawai lena yaad hai na? Jaldi se le lijiye!"
)

print("\n📋 Updated Schedule:")
display(agent.view_reminders())

---

## 🗑️ Step 8: Demonstrate Deleting a Reminder

In [ ]:
print("🗑️ DELETING REMINDER DEMONSTRATION\n")

# Delete reminder ID 5 (Vitamin supplement)
agent.delete_reminder(reminder_id=5)

print("\n📋 Updated Schedule:")
display(agent.view_reminders())

---

## 🔊 Step 9: Test Text-to-Speech Generation

In [ ]:
print("🎤 TTS GENERATION TEST\n")

# Test TTS with a sample message
test_message = "Namaste! Aapki dawai ka time ho gaya hai. Kripya apni dawai le lijiye."

print(f"Message: {test_message}\n")

audio_file = agent.generate_tts(test_message, "test_reminder.mp3")

if audio_file and AUDIO_AVAILABLE:
    print("\n🔊 Playing audio...")
    display(Audio(audio_file))
else:
    print("\n⚠️ Audio playback not available in this environment.")
    print("   In a mobile app, this would play automatically!")

---

## 💾 Step 10: Export/Import Schedule

In [ ]:
# Export current schedule
agent.export_schedule("my_medicine_schedule.json")

print("\n📁 Schedule has been saved!")
print("   You can backup this file or share it across devices.")

---

## 📊 Step 11: Summary Statistics & Analytics

In [ ]:
print("📊 MEDICINE REMINDER STATISTICS\n")
print("=" * 60)

active_reminders = [r for r in agent.reminders if r['active']]

print(f"\n📌 Total Reminders Created: {agent.reminder_id_counter - 1}")
print(f"✅ Active Reminders: {len(active_reminders)}")
print(f"🗑️ Deleted Reminders: {len(agent.reminders) - len(active_reminders)}")

if active_reminders:
    print("\n⏰ Reminder Times Distribution:")
    times = [r['time'] for r in active_reminders]
    for t in sorted(set(times)):
        count = times.count(t)
        print(f"   {t} - {count} reminder(s)")
    
    print("\n💊 Medicines Being Tracked:")
    for i, reminder in enumerate(active_reminders, 1):
        print(f"   {i}. {reminder['medicine_name']} at {reminder['time']}")

print("\n" + "=" * 60)

---

## 🎯 Step 12: Interactive User Input (Optional)

Uncomment and run this cell to add your own custom reminder:

In [ ]:
# Uncomment below to add your own reminder interactively

# print("➕ ADD YOUR OWN REMINDER\n")
# print("Please enter the following details:\n")

# medicine_name = input("Medicine Name: ")
# reminder_time = input("Reminder Time (HH:MM format, e.g., 14:30): ")
# custom_message = input("Your Custom Message (in any language): ")

# agent.add_reminder(
#     medicine_name=medicine_name,
#     reminder_time=reminder_time,
#     custom_message=custom_message
# )

# print("\n✅ Your reminder has been added!")
# display(agent.view_reminders())

print("💡 Tip: Uncomment the code above to add your own custom reminder!")

---

## 🚀 Future Enhancements & Mobile App Roadmap

### 📱 Mobile App Features (Planned)

1. **Native Mobile Notifications**
   - Background service for continuous monitoring
   - Push notifications at scheduled times
   - Snooze and "Mark as Taken" options

2. **Enhanced User Interface**
   - Large, senior-friendly buttons and text
   - Visual medicine icons and colors
   - Calendar view of upcoming reminders
   - Dark mode for night-time usage

3. **Family Dashboard**
   - Track if elderly took medicine
   - Send alerts to family members for missed doses
   - Remote schedule management
   - Medicine adherence reports

4. **Smart Features**
   - Prescription photo OCR (auto-create schedule)
   - Medicine refill reminders
   - Doctor appointment tracking
   - Health metrics logging (BP, sugar levels)

5. **Multi-Device Sync**
   - Cloud backup of schedules
   - Sync across phone, tablet, smartwatch
   - Offline mode support

6. **Communication Extensions**
   - WhatsApp integration for reminders
   - SMS fallback for feature phones
   - Voice call reminders for critical medicines

### 🛠️ Technical Implementation Plan

**Framework:** Kivy (Python) or React Native

**Backend:** Firebase or local SQLite database

**TTS Engine:** 
- Android: Google TTS API
- iOS: AVSpeechSynthesizer

**Notifications:**
- Android: WorkManager + NotificationManager
- iOS: UNUserNotificationCenter

**Testing:** Pytest for logic, Appium for mobile UI

### 📈 Project Timeline

- **Phase 1** (Current): Kaggle notebook demo ✅
- **Phase 2** (Next): Basic Kivy mobile app with local notifications
- **Phase 3**: Family dashboard and cloud sync
- **Phase 4**: Advanced features (OCR, WhatsApp, health tracking)

---

## 🎓 Educational Value

This project demonstrates:
- **Object-Oriented Programming** in Python
- **Data Management** with structured storage
- **Natural Language Processing** (multilingual support)
- **Text-to-Speech** technology
- **User-Centric Design** for accessibility
- **Real-world Problem Solving** for social impact

---

## 💡 Social Impact

### Why This Matters:

1. **Health Improvement**: Better medicine adherence = better health outcomes
2. **Elderly Care**: Empowers seniors to manage their own health
3. **Family Peace of Mind**: Reduces worry for busy family members
4. **Language Inclusion**: Supports regional languages and dialects
5. **Accessibility**: Simple interface suitable for low-tech literacy

### Target Audience:
- Elderly patients with chronic conditions
- Busy professionals managing parents' health remotely
- Caregivers managing multiple patients
- Anyone who struggles with medication adherence

---

## 📝 Conclusion

This **Medicine Reminder Agent** prototype demonstrates a practical AI solution to a real Indian healthcare challenge. The agent successfully:

✅ Manages multilingual medicine schedules

✅ Generates personalized voice reminders

✅ Provides easy schedule management

✅ Simulates real-world usage scenarios

✅ Offers clear path to mobile app deployment

**Next Steps:** Convert this logic into a mobile app using Kivy framework, integrate native notifications, and deploy to Google Play Store and Apple App Store.

---

### 👨‍💻 Developer Notes

This notebook is designed to be:
- **Educational**: Clear code with extensive comments
- **Extensible**: Easy to add new features
- **Practical**: Addresses real user needs
- **Scalable**: Architecture supports mobile app conversion

---

**🙏 Thank you for exploring this Medicine Reminder Agent!**

**Remember:** Technology should serve humanity, especially our elders. 💙